In [1]:
# ***************************************************************************************************
# Here you will find a simple version of a Variational Hybrid Quantum-Classical Algorithm based on a
# Parameterized Quantum Circuit (PQC).
# ***************************************************************************************************

# ***************************************************************************************************
# Code from:
# Zickert, F. (2021). Chapter 3: The Qubit and Quantum States
# In F. Zickert, Hands-On Quantum Machine Learning With Python: Volume 1: Get Started, PyQML, 2021. 
# Available on https://www.pyqml.com/
# ***************************************************************************************************

# ***************************************************************************************************
# Modifications have been made by Gustavo Patino.
# Engineering School
# University of Antioquia.
# Medellin, Colombia
# September, 2022.
# ***************************************************************************************************

In [2]:
# Listing 3.1 Verify Qiskit version
import qiskit
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram

import numpy as np
import pandas as pd
import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt
from math import sqrt


from sklearn.metrics import recall_score, precision_score, confusion_matrix 

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
# Descomente la siguiente linea si desea ver las imágenes más grandes.
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

In [4]:
# Importing the Qiskit lab. Drive:
import sys
sys.path.insert(0, '..')

In [5]:
# Listing 3.16: Load the data
with open('data/train.npy', 'rb') as f:
    train_input = np.load(f)
    train_labels = np.load(f)

with open('data/test.npy', 'rb') as f:
    test_input = np.load(f)
    test_labels = np.load(f)

In [6]:
# REDEFINE OR IMPORT THE FUNCTIONS OF CHAPTER 2
def run(f_classify, x):
    return list(map(f_classify, x))

def specificity(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[0][1]) if (matrix[0][0]+matrix[0][1] > 0) else 0

def npv(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[1][0]) if (matrix[0][0]+matrix[1][0] > 0) else 0

def classifier_report(name, run, classify, input, labels):
    cr_predictions = run(classify, input)
    cr_cm = confusion_matrix(labels, cr_predictions)

    cr_precision = precision_score(labels, cr_predictions)
    cr_recall = recall_score(labels, cr_predictions)
    cr_specificity = specificity(cr_cm)
    cr_npv = npv(cr_cm)
    cr_level = 0.25*(cr_precision + cr_recall + cr_specificity + cr_npv)

    print('The precision score of the {} classifier is {:.2f}'
        .format(name, cr_precision))
    print('The recall score of the {} classifier is {:.2f}'
        .format(name, cr_recall))
    print('The specificity score of the {} classifier is {:.2f}'
        .format(name, cr_specificity))
    print('The npv score of the {} classifier is {:.2f}'
        .format(name, cr_npv))
    print('The information level is: {:.2f}'
        .format(cr_level))
#CAPTION A reusable function to unmask the hypocrite classifier

In [7]:
## The Random PQC

In [8]:
# Listing 3.20: Pre‐processing template
def pre_process(passenger):
    """
    passenger -- the normalized (array of numeric data) passenger data
    returns a valid quantum state
    """
    quantum_state = [1/sqrt(2), 1/sqrt(2)]
    
    
    return quantum_state

In [9]:
# Listing 3.21: The parameterized quantum circuit (PQC)
def pqc(backend, quantum_state):
    """
    backend -- a qiskit backend to run the quantum circuit at
    quantum_state -- a valid quantum state vector  
    returns the counts of the measurement
    """

    # Create a quantum circuit with one qubit
    qc = QuantumCircuit(1)

    # Define state |Psi> and initialize the circuit
    qc.initialize(quantum_state, 0)
    
    # Measure the qubit
    qc.measure_all()

    # run the quantum circuit
    result=execute(qc,backend).result()

    # get the counts, these are either {'0': 1} or {'1': 1}
    counts=result.get_counts(qc)

    return counts

In [10]:
# Listing 3.22: Post‐processing
def post_process(counts):
    """
    counts -- the result of the quantum circuit execution
    returns the prediction
    """
    return int(list(map(lambda item: item[0], counts.items()))[0])

In [11]:
# Listing 3.23: The scores of the random quantum classifier
# Tell Qiskit how to simulate our circuit
backend = Aer.get_backend('statevector_simulator') 

In [12]:
classifier_report(
    "Random PQC",
    run,
    lambda passenger: post_process(pqc(backend, pre_process(passenger))),
    train_input,
    train_labels)

The precision score of the Random PQC classifier is 0.38
The recall score of the Random PQC classifier is 0.52
The specificity score of the Random PQC classifier is 0.48
The npv score of the Random PQC classifier is 0.62
The information level is: 0.50


In [13]:
################################################
## Variational Hybrid Quantum-Classical Algorithm with Passenger Information
################################################

In [14]:
# Listing 3.24: weigh a passenger's feature
def weigh_feature(feature, weight):
    """
    feature -- the single value of a passenger's feature
    weight -- the overall weight of this feature
    returns the weighted feature 
    """
    return feature*weight

In [15]:
# Listing 3.25: Calculate the overall probability
from functools import reduce

def get_overall_probability(features, weights):
    """
    features -- list of the features of a passenger
    weights -- list of all features' weights
    """
    return reduce(lambda result, data: result + weigh_feature(*data), zip(features, weights),0) #result = result + weigh_feature(features, weights), starting at 0.

In [16]:
# separate the training data into a list of the columns
columns = [list(map(lambda passenger: passenger[i], train_input)) for i in range(0,7)]
#columns

In [17]:
# Listing 3.26: Calculate the correlation coefficients
from scipy.stats import spearmanr

# calculate the correlation coefficient for each column
correlations = list(map(lambda col: spearmanr(col, train_labels)[0], columns))
correlations

[-0.3483505110773147,
 -0.5441105029383182,
 -0.01664108694033656,
 0.1078457168830655,
 0.12308463286494013,
 0.3269395039569518,
 -0.1916827740135955]

In [18]:
dcorr = pd.DataFrame(correlations,index =['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'], columns = ['Ranking Correlation = Feature Weight'])
dcorr

,Ranking Correlation = Feature Weight
Pclass,-0.348351
Sex,-0.544111
Age,-0.016641
SibSp,0.107846
Parch,0.123085
Fare,0.326940
Embarked,-0.191683


In [19]:
# Listing 3.27: The weighting pre‐processing
from math import pi, sin, cos

def get_state (theta):
    """returns a valid state vector from angle theta"""
    return [cos(theta/2), sin(theta/2)]

In [20]:
def pre_process_weighted(passenger):
    """
    passenger -- the normalized (array of numeric data) passenger data
    returns a valid quantum state
    """

    # caluclate the overall probability
    mu = get_overall_probability(passenger, correlations)
    print('Angle mu =', mu)
        
    # theta between 0 (|0>) and pi (|1>)
    quantum_state = get_state((1-mu)*pi)

    return quantum_state

In [21]:
backend = Aer.get_backend('statevector_simulator') 

In [22]:
# Listing 3.28: Run the PQC with the weighted pre‐processing

classifier_report("Variational Hybrid Quantum Circuit with Data Train", 
    run,
    lambda passenger: post_process(pqc(backend, pre_process_weighted(passenger))),
    train_input,
    train_labels)

Angle mu = -0.9072308497187229
Angle mu = -0.46481214951206384
Angle mu = -0.4269306339310004
Angle mu = -0.33175400091689317
Angle mu = -0.4085043078745657
Angle mu = 0.04904214788703966
Angle mu = -0.8969976592085727
Angle mu = -0.48788645159775984
Angle mu = -0.9071533168759616
Angle mu = 0.10087132455966905
Angle mu = -0.3425911095499906
Angle mu = -1.0879568438394602
Angle mu = -0.8548478604404612
Angle mu = -0.9899779776556682
Angle mu = -0.3637476810455595
Angle mu = -1.0597743680716945
Angle mu = -0.4430491287247045
Angle mu = -0.45190164111247155
Angle mu = -0.9885020449125361
Angle mu = -0.4425529050973378
Angle mu = -1.070841467133318
Angle mu = -0.4435606088442612
Angle mu = -1.065319811803422
Angle mu = 0.06464877409010415
Angle mu = -0.8911002791336133
Angle mu = -1.0838487186224766
Angle mu = -1.0829904254891745
Angle mu = -0.6670715792043456
Angle mu = -1.0895835892699668
Angle mu = -0.8968645747940646
Angle mu = -0.3279347402686018
Angle mu = -0.505261453240863
Angle m

In [23]:
# Listing 3.29: Test the PQC‐based classifier on data it has not seen before

classifier_report("Variational HQC with Data Test", 
    run,
    lambda passenger: post_process(pqc(backend, pre_process_weighted(passenger))),
    test_input,
    test_labels)

Angle mu = -0.36346103678107977
Angle mu = -0.5243772130910014
Angle mu = -1.0838207613483348
Angle mu = -0.3035696977183692
Angle mu = -0.9103675210444964
Angle mu = -0.9894754397184081
Angle mu = -0.44354979969702213
Angle mu = -0.2973981225920528
Angle mu = -0.7192498591613101
Angle mu = -0.44375891111874033
Angle mu = -1.0870204975922992
Angle mu = -0.539937038229735
Angle mu = -1.0905245906676988
Angle mu = -0.724961242702234
Angle mu = -1.0849831379921657
Angle mu = -0.5405341801369603
Angle mu = -1.084517206545052
Angle mu = -0.9710411368693186
Angle mu = -0.89395842100813
Angle mu = -0.8939611012104621
Angle mu = -1.0568356852153142
Angle mu = -1.0830122729356038
Angle mu = -0.4357899119159483
Angle mu = -1.0854811287621513
Angle mu = -0.8960281291071917
Angle mu = -1.0847265440687417
Angle mu = -0.36355280839646087
Angle mu = -0.1452949522688577
Angle mu = -0.4454340477116522
Angle mu = -1.085215808623024
Angle mu = -0.9079897625628344
Angle mu = -0.32228873188220963
Angle mu 

In [24]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright